In [1]:
in_features = 32
out_features = 8


In [2]:
from torch.nn import Linear

classical = Linear(in_features, out_features)
classical.weight, classical.bias


(Parameter containing:
 tensor([[ 0.1513, -0.0389,  0.1687,  0.0349, -0.1164, -0.0033,  0.0095, -0.1475,
           0.1628, -0.0405,  0.1569, -0.0154, -0.0783, -0.1448,  0.1442,  0.0269,
           0.0571, -0.1700,  0.1089,  0.1220, -0.0090, -0.0050, -0.0158,  0.1580,
          -0.1757,  0.1742, -0.0705,  0.1285, -0.0969, -0.0182,  0.0689, -0.0204],
         [-0.1291,  0.1670,  0.1477, -0.1322, -0.1211, -0.0441, -0.1219,  0.1027,
          -0.0129, -0.0116,  0.1102, -0.0169,  0.1627, -0.0069, -0.1022,  0.0124,
          -0.0481,  0.0207, -0.1577, -0.1467, -0.0311,  0.0491,  0.0452,  0.0696,
          -0.0294, -0.0902, -0.0552,  0.1085, -0.1381, -0.1273,  0.0828,  0.0507],
         [ 0.1741, -0.1677,  0.0723,  0.0743,  0.0354, -0.1268,  0.0203, -0.1334,
          -0.0973, -0.1454,  0.0155, -0.0670,  0.0295,  0.0008,  0.0477, -0.0699,
          -0.0286,  0.1509,  0.0728,  0.0390, -0.0445, -0.0005,  0.0225,  0.1637,
          -0.0837,  0.0490, -0.1259,  0.1600,  0.0219,  0.0322, -0.0630, 

In [3]:
from qcc.quantum.pennylane.ansatz.fully_connected import FullyConnected
from qcc.quantum.pennylane.c2q import ConvolutionFilter 

quantum = FullyConnected.from_dims(in_features, out_features, U_filter=ConvolutionFilter)
quantum.weight, quantum.bias


(Parameter containing:
 tensor([0.6922, 6.1755, 2.0536, 0.4415, 2.4870, 4.7588, 2.1313, 0.7777, 2.1181,
         3.4901, 5.8533, 3.9499, 1.5229, 3.7413, 6.2067, 4.4179, 4.2624, 4.8898,
         5.1127, 1.0826, 0.3721, 5.8603, 1.0734, 2.5090, 5.0157, 4.2063, 1.2370,
         0.0681, 4.5753, 4.7051, 2.6940, 2.8808, 5.6310, 1.5579, 4.5407, 0.1777,
         5.1019, 3.6090, 1.5825, 3.0374, 0.5733, 4.0572, 5.9597, 1.6700, 0.9190,
         2.8603, 3.6177, 2.4843, 2.2964, 0.6688, 5.6951, 0.9304, 5.8388, 4.0105,
         4.8461, 3.6462, 2.9529, 6.1268, 2.9129, 2.8119, 1.5548, 4.1798, 3.8909,
         5.3295, 3.9538, 1.7449, 4.1877, 5.0597, 0.5994, 5.3571, 4.6058, 4.9971,
         4.2458, 6.2021, 1.4549, 2.6197, 4.5870, 1.8192, 3.0402, 4.5668, 3.0834,
         5.2847, 2.3816, 0.4134, 1.6715, 3.5202, 1.0501, 1.0092, 1.4187, 3.6385,
         0.7106, 2.8018, 2.6465, 5.7463, 4.4491, 4.1844, 3.2418, 2.8491, 4.3365,
         4.3832, 6.2304, 6.1838, 3.4210, 0.5839, 2.7858, 4.0276, 0.2954, 1.0254,
     

In [4]:
import torch

weight = torch.abs(classical.weight.T / torch.norm(classical.weight, dim=1)).T
bias = classical.bias

weight = torch.nn.Parameter(weight)
bias = torch.nn.Parameter(bias)

classical.weight, classical.bias = weight, bias
quantum.weight, quantum.bias = torch.nn.Parameter(weight.flatten()), bias

assert((classical.weight.flatten() == quantum.weight).all())
assert((classical.bias == quantum.bias).all())


In [5]:
data = torch.nn.init.uniform_(torch.zeros(in_features))
data = data / torch.abs(data)


In [6]:
classical.weight @ data


tensor([4.6498, 4.8127, 4.6451, 5.0606, 4.6992, 4.7848, 4.8809, 5.3310],
       grad_fn=<MvBackward0>)

In [7]:


classical(data), quantum(data)


(tensor([4.4870, 4.7840, 4.6724, 4.9716, 4.5248, 4.9218, 4.8586, 5.3486],
        grad_fn=<ViewBackward0>),
 tensor([[4.4870, 4.7840, 4.6724, 4.9716, 4.5248, 4.9218, 4.8586, 5.3486]],
        dtype=torch.float64, grad_fn=<AddBackward0>))